# <font color="#1b5776">1. <u>Adding Imports</u></font>

In [ ]:
import pandas as pd
import numpy as np

import pickle

import warnings
warnings.filterwarnings("ignore")

# <font color="#1b5776">2. <u>Getting the Data</u></font>

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/train_cleaned2.csv",dtype={"fullVisitorId":"string"})

test = pd.read_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/test_cleaned2.csv",dtype={"fullVisitorId":"string"})

In [ ]:
train.head(2)

,fullVisitorId,channelGrouping,date,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,totals_visits,totals_hits,totals_pageviews,totals_bounces,totals_newVisits,totals_sessionQualityDim,totals_timeOnSite,totals_transactions,totals_transactionRevenue,totals_totalTransactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_campaignCode,index,value
0,3162355547410993243,Organic Search,20171016,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,(not set),google,organic,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,EMEA
1,8934116514970143966,Referral,20171016,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,Chrome OS,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2.0,NaN,NaN,2.0,28.0,NaN,NaN,NaN,(not set),sites.google.com,referral,NaN,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,North America


In [ ]:
test.head(2)

,fullVisitorId,channelGrouping,date,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_sessionQualityDim,totals_newVisits,totals_transactions,totals_transactionRevenue,totals_totalTransactionRevenue,totals_bounces,trafficSource_referralPath,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_isTrueDirect,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,index,value
0,7460955084541987166,Organic Search,20180511,Not Socially Engaged,1526099341,2,1526099341,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Asia,Southern Asia,India,Delhi,(not set),(not set),not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,4,3.0,973.0,1,NaN,NaN,NaN,NaN,NaN,(not set),(not set),google,organic,(not provided),(not set),True,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,4.0,APAC
1,460252456180441002,Direct,20180511,Not Socially Engaged,1526064483,166,1526064483,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,4,3.0,49.0,1,NaN,NaN,NaN,NaN,NaN,(not set),(not set),(direct),(none),(not set),(not set),True,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,4.0,North America


Initializing a few lists/dictionaries to store relevant data which will be used later on in creating production pipeline.

In [ ]:
drop_columns = []
impute_columns_with_zero = []
impute_columns_with_OTHERS = []

# <font color="#1b5776">3. <u>Data Cleaning</u></font>

In [ ]:
print("train shape:",train.shape)
print("test shape:",test.shape)

train shape: (1708337, 60)
test shape: (401589, 59)


Observation: test file has 1 column less than the train file.
Let's examine this a bit more.

In [ ]:
# checking if train and test sets have same columns or not:-
for col in train.columns:
  if col not in test.columns:
    print(col)
    drop_columns.append(col)

trafficSource_campaignCode


The column <b>trafficSource_campaignCode</b> is present only in the train set. Hence, we need to drop the column: <b>trafficSource_campaignCode</b> from the train set.

In [ ]:
train.drop(columns=['trafficSource_campaignCode'],axis=1,inplace=True)

In [ ]:
print("train shape:",train.shape)
print("test shape:",test.shape)

train shape: (1708337, 59)
test shape: (401589, 59)


Now, both train & test datasets have the same 59 columns. Let's proceed with feature-wise cleaning wherever & whenever required. ✅

In [ ]:
def about_data(df):
  """
  This function takes in a dataframe and returns a dataframe with the following details about the input dataframe's columns:-
  * No. of unique values
  * Percentage missing data (eg:nan,NaN,none,null etc.)
  * Percentage of abrupt data (eg:"(not set)", "not available in demo dataset", "unknown.unknown" etc.) present
  """
  unique_counts = []
  null_pct = []
  abrupt_pct = []

  for col in df.columns:
    total_uniques = df[col].unique()
    unique_counts.append(len(total_uniques))

    null_count = df[col].isna().sum()
    null_pct.append((null_count/df.shape[0])*100)

    abrupt_data_count = df[col].isin(["(not set)","not available in demo dataset","(none)","(not provided)","unknown.unknown"]).sum()
    abrupt_pct.append((abrupt_data_count/df.shape[0])*100)

  about_data_df = pd.DataFrame(data={"column":df.columns,
                                     "unique_counts":unique_counts,
                                     "null_data_%":null_pct,
                                     "abrupt_data_%":abrupt_pct,
                                     "data_type":train.dtypes.values})
  print("We have",len(df.columns),"columns now.")
  print()
  return about_data_df

I'll go step-by-step with the Data Cleaning process. And after every step of cleaning is complete, I'll display the status of the dataframe again using this about_data function. ♻ </br>
1 thing to note is: whatever cleaning is done on the train file, it has to be done on test file also. So, I'll not run this function for the test file.

<font color="#7fde25"><b><u>1st level of cleaning:-</u><b></font>

In [ ]:
about_train_df = about_data(train)
about_train_df.sort_values(by="unique_counts",ascending=True)

We have 59 columns now.



,column,unique_counts,null_data_%,abrupt_data_%,data_type
29,geoNetwork_cityId,1,0.000000,100.000000,object
32,geoNetwork_longitude,1,0.000000,100.000000,object
33,geoNetwork_networkLocation,1,0.000000,100.000000,object
34,totals_visits,1,0.000000,0.000000,int64
21,device_screenResolution,1,0.000000,100.000000,object
20,device_screenColors,1,0.000000,100.000000,object
19,device_language,1,0.000000,100.000000,object
18,device_flashVersion,1,0.000000,100.000000,object
17,device_mobileDeviceMarketingName,1,0.000000,100.000000,object
16,device_mobileDeviceInfo,1,0.000000,100.000000,object


Note: There are many columns with just 1 type of value. There's no point in keeping these features in the dataset, as they won't be adding any value for our ultimate goal, i.e. prediction. (Let's not forget our problem at hand, right? 😳 )

In [ ]:
for col in train.columns:
  if len(train[col].unique()) == 1:
    drop_columns.append(col)
    train.drop(columns=[col],axis=1,inplace=True)
    test.drop(columns=[col],axis=1,inplace=True)

<font color="#7fde25"><b><u>2nd level of cleaning:-</u><b></font>


In [ ]:
about_train_df = about_data(train)
about_train_df.sort_values(by="unique_counts",ascending=True)

We have 40 columns now.



,column,unique_counts,null_data_%,abrupt_data_%,data_type
19,totals_bounces,2,49.192343,0.000000,float64
37,trafficSource_adwordsClickInfo.isVideoAd,2,92.139666,0.000000,object
31,trafficSource_isTrueDirect,2,68.569316,0.000000,object
8,device_isMobile,2,0.000000,0.000000,bool
20,totals_newVisits,2,24.887712,0.000000,float64
38,index,2,19.506397,0.000000,float64
36,trafficSource_adwordsClickInfo.adNetworkType,3,92.139666,0.000000,object
34,trafficSource_adwordsClickInfo.slot,3,92.139666,0.000000,object
23,totals_transactions,3,99.120314,0.000000,float64
9,device_deviceCategory,3,0.000000,0.000000,object


Now let's investigate the columns having 2 unique values. They may have <b>nan</b> as the other value.

In [ ]:
for col in train.columns:
  u = train[col].unique()
  if len(u) == 2:
    print("Column:",col,"-->","unique values:",u)

Column: device_isMobile --> unique values: [False  True]
Column: totals_bounces --> unique values: [ 1. nan]
Column: totals_newVisits --> unique values: [ 1. nan]
Column: trafficSource_isTrueDirect --> unique values: [nan True]
Column: trafficSource_adwordsClickInfo.isVideoAd --> unique values: [nan False]
Column: index --> unique values: [ 4. nan]


Commenting on the above 6 columns:-

<b>device_isMobile</b>:-<br>It column has 2 different values and has no missing/abrupt data. Hence, I'll leave this column as is. In fact, this feature can actually help us a lot in prediction!

<b>totals_bounces</b><br>
A 'bounce' (often called a single-page session) happens when a user lands on a website page and exits without triggering another request to the Google Analytics server. (Source: [Google Search](https://www.google.com/search?q=bounced+session&rlz=1C1UEAD_enIN959IN959&oq=bounced+s&aqs=chrome.1.69i57j69i59j0i512l4j0i10i512l4.3219j0j7&sourceid=chrome&ie=UTF-8#:~:text=What%20is%20a%20bounced%20session%3F)).<br>
Also, as per [this documentation](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=totals.bounces,it%20is%20null.)
the value can only be <b>1</b>(indicating bounced session) otherwise <b>null</b>.
And since we don't have the idea whether the 49% nan values indicate absence of data or non-bounced session, we cannot assume any specific value.
Also, imputing with any other value won't add much value to this feature, as it can take only 1 type of value in reality.
Hence, let's just drop this column.

<b>totals_newVisits</b><br>
As per [this documentation](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=totals.newVisits,it%20is%20null.) the value can only be <b>1</b>(indicating first visit) otherwise <b>null</b>. And since we don't have the idea whether the 24% nan values
indicate absence of data or non-first-session, we cannot assume any specific value.
Also, imputing with any other value won't add much value to this feature, as it can take only 1 type of value in reality.
Hence, let's just drop this column.

<b>trafficSource_isTrueDirect</b><br>
As per [this documentation](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=trafficSource.isTrueDirect,details.%20Otherwise%20NULL.) the value can only be <b>True</b>(indicating first visit) otherwise <b>null</b>. And since we don't have the idea whether the 68% nan values
indicate absence of data or non-direct session, we cannot assume any specific value.
Also, imputing with any other value won't add much value to this feature, as it can take only 1 type of value in reality.
Hence, let's just drop this column.

<b>trafficSource_adwordsClickInfo.isVideoAd</b><br>
As per [this documentation](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=True%20if%20it%20is%20a%20Trueview%20video%20ad.) it can take only <br>True</b> as values. But we've been given either False or nan. There's no point of keeping such an abrupt feature. It won't add any value as such.
Also, let's just drop it as it has above 92% nan values.

<b>index</b>:-<br>
This was obtained after processing the customDimensions column present initially. Only the value 4 or nan is present in the dataset. Let's just drop this column

In [ ]:
# dropping all 5 except device_isMobile:-
for col in train.columns:
  if len(train[col].unique())==2 and col != "device_isMobile":
    drop_columns.append(col)
    train.drop(columns=[col],axis=1,inplace=True)
    test.drop(columns=[col],axis=1,inplace=True)

<font color="#7fde25"><b><u>3rd level of cleaning:-</u><b></font>



In [ ]:
about_train_df = about_data(train)
about_train_df.sort_values(by="null_data_%",ascending=False)

We have 35 columns now.



,column,unique_counts,null_data_%,abrupt_data_%,data_type
23,totals_totalTransactionRevenue,73,99.148763,0.000000,float64
22,totals_transactionRevenue,70,99.148763,0.000000,float64
21,totals_transactions,3,99.120314,0.000000,float64
29,trafficSource_adContent,15,93.881008,0.000000,object
33,trafficSource_adwordsClickInfo.adNetworkType,3,92.139666,0.000000,object
31,trafficSource_adwordsClickInfo.slot,3,92.139666,0.000000,object
30,trafficSource_adwordsClickInfo.page,4,92.139666,0.000000,float64
32,trafficSource_adwordsClickInfo.gclId,359,92.129480,0.000000,object
28,trafficSource_referralPath,244,73.022477,0.000000,object
20,totals_timeOnSite,908,50.925491,0.000000,float64


Let's now look into the features that are left and impute them as per requirement:-

<b>totals_transactionRevenue</b>, <b>totals_totalTransactionRevenue</b></br>
I went through some Kaggle discussions like [this](https://www.kaggle.com/c/ga-customer-revenue-prediction/discussion/71779) and [this](https://www.kaggle.com/c/ga-customer-revenue-prediction/discussion/71260) from Kaggle's discussion page of this competition. And although [this](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=totals.transactionRevenue,instead%20(see%20above).) description (from Google Analytics page) about <b>transactionRevenue</b> says that it is deprecated, the competition was evaluated on <b>transactionRevenue</b> and not <b>totalTransactionRevenue</b>. So, we have to work using <b>transactionRevenue</b> only. Hence, it's better to drop <b>totalTransactionRevenue</b> as it will lead to data leakage if it is kept as a feature (since <b>totalTransactionRevenue</b> contains <b>transactionRevenue</b> from 1 transaction done by the visitor).


In [ ]:
drop_columns.append("totals_totalTransactionRevenue")

In [ ]:
train.drop(columns=["totals_totalTransactionRevenue"],axis=1,inplace=True)
test.drop(columns=["totals_totalTransactionRevenue"],axis=1,inplace=True)

In [ ]:
print("train shape",train.shape)
print("test shape",test.shape)

train shape (1708337, 34)
test shape (401589, 34)


<b>totals_transactions</b></br>
Referring [this documentation](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=totals.transactions,within%20the%20session.) let's impute nan with 0 --> indicating that the user performed 0 ecommerce transactions within the session.

<b>trafficSource_adwordsClickInfo.adNetworkType</b></br>
As per [this](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=Network%20Type.%20Takes%20one%20of%20the%20following%20values%3A%20%7B%E2%80%9CGoogle%20Search%22%2C%20%22Content%22%2C%20%22Search%20partners%22%2C%20%22Ad%20Exchange%22%2C%20%22Yahoo%20Japan%20Search%22%2C%20%22Yahoo%20Japan%20AFS%22%2C%20%E2%80%9Cunknown%E2%80%9D%7D) this feature can take multiple other values as well. Hence, let's just impute nan with OTHERS.

<b>trafficSource_adwordsClickInfo.slot</b></br>
As per [this](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=Position%20of%20the%20Ad.%20Takes%20one%20of%20the%20following%20values%3A%7B%E2%80%9CRHS%22%2C%20%22Top%22%7D) the value can only be RHS or TOP. And it has 92% null values. It's better to just drop it.

<b>trafficSource_adwordsClickInfo.page	</b></br>
This feature is currently of float64 data type. Referring [this documentation](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=Page%20number%20in%20search%20results%20where%20the%20ad%20was%20shown.) imputing nan with 0 will mean --> the ad was shown in Page Number 0. Although it has 92% nan values, it is better to keep it since it conveys info on where the ad was shown.

<b>trafficSource_adwordsClickInfo.gclId</b></br>
This is simply a Google Click Id. Refer [this](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=The%20Google%20Click%20ID.), with 92% nan values. It is better to drop it.

<b>totals_timeOnSite</b></br>
As per [this](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=totals.timeOnSite,expressed%20in%20seconds.) the value are in seconds. Let's impute nan with 0 --> indicating that the user spent 0 seconds in the session.

<b>totals_sessionQualityDim</b></br>
As per [this](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=totals.sessionQualityDim,selected%20time%20range.) the value can range from 1 to 100. And a value of 0 will indicate that Session Quality is not calculated for the selected time range.
Also, note: there is an ordinal nature to this feature.

<b>totals_pageviews</b></br>
Referring [this](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=totals.pageviews,within%20the%20session.) imputing with nan with 0 --> indicating 0 pageviews within the session

<b>visitId</b></br>
Referring [this](https://support.google.com/analytics/answer/3437719?hl=en#:~:text=An%20identifier%20for%20this%20session.%20This%20is%20part%20of%20the%20value%20usually%20stored%20as%20the%20_utmb%20cookie.%20This%20is%20only%20unique%20to%20the%20user.%20For%20a%20completely%20unique%20ID%2C%20you%20should%20use%20a%20combination%20of%20fullVisitorId%20and%20visitId.) visitId is an just identifier of the session. Hence, dropping it from both: train & test datasets

In [ ]:
to_replace_dict = {"(not set)":"OTHERS","not available in demo dataset":"OTHERS","(none)":"OTHERS","(not provided)":"OTHERS","unknown.unknown":"OTHERS"}
for col in train.columns:
  if col in ["visitId","trafficSource_adwordsClickInfo.slot","trafficSource_adwordsClickInfo.gclId"]:
    drop_columns.append(col)
    train.drop(columns=[col],axis=1,inplace=True)
    test.drop(columns=[col],axis=1,inplace=True)

  elif train[col].dtype == "float64":
    impute_columns_with_zero.append(col)
    train[col].fillna(value=0,inplace=True)
    test[col].fillna(value=0,inplace=True)

  elif train[col].dtype == "object":
    impute_columns_with_OTHERS.append(col)
    # imputing nan with OTHERS
    train[col].fillna(value="OTHERS",inplace=True)
    test[col].fillna(value="OTHERS",inplace=True)
    # imputing '(not set)','not available in demo dataset','(none)','(not provided)','unknown.unknown' values with "OTHERS":-
    # referance: https://stackoverflow.com/questions/40273067/replace-specific-values-in-a-dataframe-column-using-pandas
    train[col] = train[col].replace(to_replace=to_replace_dict)
    test[col] = test[col].replace(to_replace=to_replace_dict)

In [ ]:
about_train_df = about_data(train)
about_train_df.sort_values(by="column",ascending=False)

We have 31 columns now.



,column,unique_counts,null_data_%,abrupt_data_%,data_type
4,visitStartTime,1667423,0.0,0.0,int64
3,visitNumber,457,0.0,0.0,int64
30,value,6,0.0,0.0,object
23,trafficSource_source,52,0.0,0.0,object
26,trafficSource_referralPath,244,0.0,0.0,object
24,trafficSource_medium,6,0.0,0.0,object
25,trafficSource_keyword,58,0.0,0.0,object
22,trafficSource_campaign,11,0.0,0.0,object
28,trafficSource_adwordsClickInfo.page,4,0.0,0.0,float64
29,trafficSource_adwordsClickInfo.adNetworkType,3,0.0,0.0,object


Observations: As we can now see, all suitable columns have been imputed and some have been dropped. We are no longer left with any null or irrelevant value in our datasets. We are now left with 31 columns which we can take further for EDA and modelling.

In [ ]:
# clean train dataset
train.head(2)

,fullVisitorId,channelGrouping,date,visitNumber,visitStartTime,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain,totals_hits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_referralPath,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.adNetworkType,value
0,3162355547410993243,Organic Search,20171016,1,1508198450,Firefox,Windows,False,desktop,Europe,Western Europe,Germany,OTHERS,OTHERS,OTHERS,OTHERS,1,1.0,1.0,0.0,0.0,0.0,OTHERS,google,organic,water bottle,OTHERS,OTHERS,0.0,OTHERS,EMEA
1,8934116514970143966,Referral,20171016,6,1508176307,Chrome,Chrome OS,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,OTHERS,2,2.0,2.0,28.0,0.0,0.0,OTHERS,sites.google.com,referral,OTHERS,/a/google.com/transportation/mtv-services/bike...,OTHERS,0.0,OTHERS,North America


In [ ]:
# clean test dataset
test.head(2)

,fullVisitorId,channelGrouping,date,visitNumber,visitStartTime,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain,totals_hits,totals_pageviews,totals_timeOnSite,totals_sessionQualityDim,totals_transactions,totals_transactionRevenue,trafficSource_referralPath,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.adNetworkType,value
0,7460955084541987166,Organic Search,20180511,2,1526099341,Chrome,Android,True,mobile,Asia,Southern Asia,India,Delhi,OTHERS,OTHERS,OTHERS,4,3.0,973.0,1,0.0,0.0,OTHERS,OTHERS,google,organic,OTHERS,OTHERS,0.0,OTHERS,APAC
1,460252456180441002,Direct,20180511,166,1526064483,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,OTHERS,4,3.0,49.0,1,0.0,0.0,OTHERS,OTHERS,(direct),OTHERS,OTHERS,OTHERS,0.0,OTHERS,North America


The sequence of columns in test is a bit different than train set. The column <b>totals_timeOnSite</b> and <b>totals_sessionQualityDim</b> are in different orders. Let's make them same as train set.

In [ ]:
train_columns_sequence = list(train.columns)

In [ ]:
test = test[train.columns]

In [ ]:
test.head(2)

,fullVisitorId,channelGrouping,date,visitNumber,visitStartTime,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain,totals_hits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_referralPath,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.adNetworkType,value
0,7460955084541987166,Organic Search,20180511,2,1526099341,Chrome,Android,True,mobile,Asia,Southern Asia,India,Delhi,OTHERS,OTHERS,OTHERS,4,3.0,1,973.0,0.0,0.0,OTHERS,google,organic,OTHERS,OTHERS,OTHERS,0.0,OTHERS,APAC
1,460252456180441002,Direct,20180511,166,1526064483,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,OTHERS,4,3.0,1,49.0,0.0,0.0,OTHERS,(direct),OTHERS,OTHERS,OTHERS,OTHERS,0.0,OTHERS,North America


We are now good to go! 😆 Let's save these clean files now.

In [ ]:
train.to_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/train_cleaned3.csv",index=False)

test.to_csv("/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/test_cleaned3.csv",index=False)

In [ ]:
# storing all the lists/dicts to dictionaries to be used later on:-

f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/drop_columns.pkl"
with open(f_path,"wb") as f:
  pickle.dump(drop_columns,f)
f.close()

f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/impute_columns_with_zero.pkl"
with open(f_path,"wb") as f:
  pickle.dump(impute_columns_with_zero,f)
f.close()

f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/impute_columns_with_OTHERS.pkl"
with open(f_path,"wb") as f:
  pickle.dump(impute_columns_with_OTHERS,f)
f.close()

f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/to_replace_dict.pkl"
with open(f_path,"wb") as f:
  pickle.dump(to_replace_dict,f)
f.close()

f_path = "/content/drive/MyDrive/AAIC - Assignments/SNo.23_Self Case Study 1/Output_Files/train_columns_sequence.pkl"
with open(f_path,"wb") as f:
  pickle.dump(train_columns_sequence,f)
f.close()